## Metaclasses

To create metaclass we need to inherit from type or any child class which is inherited from type. Metaclass is used to build any class in python. 

![](uml/metaclass.png)

### Example metaclass
In this example we will create a metaclass from abc.ABCMeta, which inherits from type. Will will than decorate every `roll()` method in child class inherited from interface which uses this metaclass.

In [10]:
import abc

print(abc.ABCMeta.__mro__)

(<class 'abc.ABCMeta'>, <class 'type'>, <class 'object'>)


In [12]:
import logging
from functools import wraps
from typing import Type, Any, cast
import sys
import random
import time


class DieMeta(abc.ABCMeta):
    def __new__(
        metaclass: Type[type],
        name: str,
        bases: tuple[type, ...],
        namespace: dict[str, Any],
        **kwargs: Any,
    ) -> "DieMeta":
    
        if "roll" in namespace and not getattr(
            namespace["roll"], "__isabstractmethod__", False):

            namespace.setdefault("logger", logging.getLogger(name))

            original_method = namespace["roll"]

            @wraps(original_method)
            def logged_roll(self: "DieLog") -> None:
                original_method(self)
                self.logger.info(f"Rolled {self.face}")

            namespace["roll"] = logged_roll
    
        new_object = cast(
            "DieMeta", abc.ABCMeta.__new__(
                metaclass, name, bases, namespace
            )
        )
        return new_object
    
    
class DieLog(metaclass=DieMeta):
    logger: logging.Logger
        
    def __init__(self) -> None:
        self.face: int
        self.roll()
        
    @abc.abstractmethod
    def roll(self) -> None: ...
        
    def __repr__(self) -> str:
        return f"{self.face}"
    
    
class D6L(DieLog):
    def roll(self) -> None:
        """This function useses logg_roll"""
        self.face = random.randrange(1, 7)
        
        
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
d2 = D6L()
time.sleep(1)
d2.face

INFO:D6L:Rolled 2


2

**Note:** 
Every class which inherits from inteface which uses our metaclass will use the same metaclass.
Also note, we check if the method which will be wrapped with our decorator is abstract.    
    

In [8]:
if __name__ == '__main__':        
    import doctest
    import subprocess
    name = '06-Metaclass'
    doctest.testmod(verbose=False)
    subprocess.run(f'jupyter nbconvert --to script --output test "{name}"', shell=True)
    std_out = subprocess.run('mypy --strict test.py', capture_output=True, shell=True).stdout
    print(std_out.decode('ascii'))

[NbConvertApp] Converting notebook 06-Metaclass.ipynb to script
[NbConvertApp] Writing 2609 bytes to test.py


Success: no issues found in 1 source file

